In [3]:
import tensorflow as tf
import numpy as np
import os
import time

In [4]:
with open("龙族.txt","r") as f: 
    text = f.read()  
    #print(text)
len(text)
type(text)

str

In [5]:
# Taking a look at the text
print(text[:300])

路明非在屏幕上无奈地打出“GG”（“GG”，指“Good Game”，在竞技类游戏中称赞对方玩得好，也是认负的意思），而后切出了游戏。屏幕上显示的最后一个场景，是十二艘人类巡洋舰以华丽的大和炮聚焦射击，把他的母巢化作了一滩血水。

他输掉了今天的第六局，胜负比例是零比六，这一次他坚持到了22分23秒才被拿下，不过最终还是被拿下了，对方的微操很好，用的又是人类，人类的机枪兵在这个游戏里是个变态的兵种，出枪速度为零，站住了拔枪就射，收枪就跑，路明非的小狗追不上，在路上就一只只被打爆了。

公共聊天频道里，对手正侃侃而谈，“人类打虫族未必要出坦克，高手都不太出坦克了，开始就爆兵，海量的机枪混着护士冲


In [58]:
strr = ['猪','是','狗','一']
stt = '输入网址'
print(type(strr))
ccc = set(strr)
#print(ccc)
#print(sorted(ccc))
list(enumerate(ccc))
print(c for c in stt)
type(stt)
ccc.batch

<class 'list'>
<generator object <genexpr> at 0x7fec2dde0bd0>


AttributeError: ignored

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

3532 unique characters


In [8]:
# Creating a mapping from unique characters to indices
char2index = {u:i for i, u in enumerate(vocab)}

index2char = np.array(vocab)

text_as_int = np.array([char2index[c] for c in text])

print(text_as_int)

[3041 1575 3352 ... 3073  837 3515]


In [55]:
index2char 

array(['\n', ' ', '"', ..., 'Ｒ', '｛', '｝'], dtype='<U1')

In [11]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} -- characters mapped to int -- > {}'.format(repr(text[:13]), text_as_int[:13]))

'路明非在屏幕上无奈地打出“' -- characters mapped to int -- > [3041 1575 3352  776 1027 1099  108 1563  863  777 1324  404   91]


In [40]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(index2char[i.numpy()])


路
明
非
在
屏


In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index2char[item.numpy()])))



'路明非在屏幕上无奈地打出“GG”（“GG”，指“Good Game”，在竞技类游戏中称赞对方玩得好，也是认负的意思），而后切出了游戏。屏幕上显示的最后一个场景，是十二艘人类巡洋舰以华丽的大和炮聚焦射击，'
'把他的母巢化作了一滩血水。\n\n他输掉了今天的第六局，胜负比例是零比六，这一次他坚持到了22分23秒才被拿下，不过最终还是被拿下了，对方的微操很好，用的又是人类，人类的机枪兵在这个游戏里是个变态的兵种，出'
'枪速度为零，站住了拔枪就射，收枪就跑，路明非的小狗追不上，在路上就一只只被打爆了。\n\n公共聊天频道里，对手正侃侃而谈，“人类打虫族未必要出坦克，高手都不太出坦克了，开始就爆兵，海量的机枪混着护士冲过去，'
'连消带打，看住对方小狗没有升级速度之前压制住了，他就只有不断出兵跟你磨，他刺蛇不能成队你就赢了，后面巡洋舰编队出击，那是压倒性的啊……”路明非可以想像那家伙眉飞色舞的样子。\n\n路明非没啃声儿，切到QQ上'
'，那个戴棒球帽的女孩头像还是灰色的，一动不动。对方没上线，又白等了。他抓了抓脑袋，有点失望。另一个头像倒是跳了起来，是个长得很欠的熊猫。\n\n“兄弟你虫族玩得不错了，下次再切！”熊猫是那个打赢了他的家伙，'


In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [66]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [64]:
a=[3,1,1,1,1,2]

In [65]:
split_input_target(a)

([3, 1, 1, 1, 1], [1, 1, 1, 1, 2])

In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300 #256

# Number of RNN units
rnn_units1 = 512 #1024
rnn_units2 = 256
rnn_units=[rnn_units1, rnn_units2]
print(vocab_size)

3532


In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [19]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 300)           1059600   
                                                                 
 gru (GRU)                   (64, None, 512)           1250304   
                                                                 
 gru_1 (GRU)                 (64, None, 256)           591360    
                                                                 
 dense (Dense)               (64, None, 3532)          907724    
                                                                 
Total params: 3,808,988
Trainable params: 3,808,988
Non-trainable params: 0
_________________________________________________________________


In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
EPOCHS=50

In [25]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
55/55 [==============================] - 10s 51ms/step - loss: 6.5497 - accuracy: 0.0487
Epoch 2/50
55/55 [==============================] - 3s 48ms/step - loss: 6.2377 - accuracy: 0.0498
Epoch 3/50
55/55 [==============================] - 3s 49ms/step - loss: 6.2381 - accuracy: 0.0499
Epoch 4/50
55/55 [==============================] - 3s 50ms/step - loss: 6.2385 - accuracy: 0.0498
Epoch 5/50
55/55 [==============================] - 3s 50ms/step - loss: 6.2395 - accuracy: 0.0498
Epoch 6/50
55/55 [==============================] - 3s 50ms/step - loss: 6.2390 - accuracy: 0.0499
Epoch 7/50
55/55 [==============================] - 3s 51ms/step - loss: 6.2384 - accuracy: 0.0499
Epoch 8/50
55/55 [==============================] - 3s 51ms/step - loss: 6.2397 - accuracy: 0.0498
Epoch 9/50
55/55 [==============================] - 3s 53ms/step - loss: 6.2362 - accuracy: 0.0498
Epoch 10/50
55/55 [==============================] - 3s 51ms/step - loss: 6.2380 - accuracy: 0.0498
Epoch 11

In [26]:
latest_check= tf.train.latest_checkpoint(checkpoint_dir)

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(latest_check)

model.build(tf.TensorShape([1, None]))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 300)            1059600   
                                                                 
 gru_2 (GRU)                 (1, None, 512)            1250304   
                                                                 
 gru_3 (GRU)                 (1, None, 256)            591360    
                                                                 
 dense_1 (Dense)             (1, None, 3532)           907724    
                                                                 
Total params: 3,808,988
Trainable params: 3,808,988
Non-trainable params: 0
_________________________________________________________________


In [29]:
def generate_text(model, start_string):

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low results in more predictable text.
  # Higher results in more surprising text.
  # Experiment to find the best setting.
  scaling = 0.5 #1

  # batch size == 1
  
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / scaling
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(index2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
print(generate_text(model, start_string=u"路明非"))

路明非子，”是他一你的来的起的一能是
”





是1









你

有
“




是是是死，是记一是他可





院影，


“”是


在大，”






是是这他他的，
“

还
明的的诺


”

是


路是不

这这不


””是
他生，
可
死的。










明，“


”一”

有


是只
是

”







“
他他”然”是说可会这
诺
是路
”
路芬
“



楚。

明的这是
””





个 





一”
“路有在


两“


明我因
你”

”他有一

有
“不


路”
““他一


个是“他的这不”了，撒，是在是



你是的，

““




是是你
”是
“明，”你

“3，那


“
明，
“

”


是，


有他一
“
是路

“
芬”


”是
声，


““





明的

“他，
是们不



明，““
是还他这他，
个的人起，一


但


他“也

授，不是
我

他非“





他在是你一


”明非




院了
可是这，

“”我一“

明，在



是明，不说“




是






”我这是

“是说这有是
你


”

”是芬他，

他他的，
“你你不
他，
”
这是这


这


”

”个。
他

是到长，



“是这他

他是是是一”是我和有那

个不明，

那一


““
“

为在一
也一


”






““是他明，
他有的的的着人，是”



“
法这是得出，




”有这
没是

明的，
真他““

恺
不有他是”也你

这“明的那


”



个看，个，“
他 你路



路是


””他你的。


是是一把们静，
个有了，


“在


我是
“


“一刚

“
他学



是
我我这“明了，
这没






我是他个会

明地”。路
”我的的，
西幺，
“


”





是他他”的，

“是，
一个看，

撒一“
”



明在
“

他

“
“
这
也明，我一
是

“


”















”个里会






””是
是手，个，


不”诺是还个不把是在”，
”

路
不人一一“
我这






明的是你一
“
明的得的在
”明，

“
他

”是事…，


有？不？明的的作这这一什，格的，